In [48]:
import pandas as pd
import numpy as np
from scipy import signal
%matplotlib notebook
import matplotlib.pyplot as plt

OVERLAP = 0.5 # in seconds
WIN_LEN = 1 # in seconds

data = pd.read_csv('../muse-datasets/raw-data/nasui-focused-1.csv')
data = data.iloc[:,:5]

data['timestamps'] = data['timestamps'] - data['timestamps'].iloc[0]
data = data.loc[(data['timestamps'] >= 3) & (data['timestamps'] < 63)]
data['timestamps'] = data['timestamps'] - data['timestamps'].iloc[0]
data.rename(columns={'timestamps':'Time'},inplace=True)
data.reset_index(drop=True,inplace=True)
data.head()

,Time,TP9,AF7,AF8,TP10
0,0.000,-224.609,-59.570,125.977,-211.426
1,0.004,-222.168,-57.129,132.324,-269.043
2,0.008,-205.078,-53.711,138.184,-340.820
3,0.012,-199.219,-52.246,138.184,-329.590
4,0.016,-207.031,-52.246,136.719,-248.047


In [2]:
start = 0
end = start + 1
w_time = data.loc[(data['Time'] >= start) & (data['Time'] < end)]
w = w_time.iloc[:,1:] if w_time.columns[0] == 'Time' else None

In [3]:
# Split window using np.split()
wh = np.vsplit(w,2)
wq = np.vsplit(w,4)
w_max = w.max()
w_max.rename(index={'TP9':'std_0','AF7':'std_1','AF8':'std_2','TP10':'std_3'}, inplace=True)


In [4]:
# Extract mean, std dev of each full window
std = w.std()
std.rename(index={'TP9':'std_0','AF7':'std_1','AF8':'std_2','TP10':'std_3'}, inplace=True)
mean = w.mean()
mean.rename(index={'TP9':'mean_0','AF7':'mean_1','AF8':'mean_2','TP10':'mean_3'}, inplace=True)

In [5]:
# Skewness
w_skew = w.skew()
w_skew.rename(index={'TP9':'skew_0','AF7':'skew_1','AF8':'skew_2','TP10':'skew_3'}, inplace=True)
w_kurtosis = w.kurtosis()
w_kurtosis.rename(index={'TP9':'kurt_0','AF7':'kurt_1','AF8':'kurt_2','TP10':'kurt_3'}, inplace=True)

In [53]:
# Signals covariance matrix
w_cov_matrix = w.cov()
w_cov_matrix['TP9'] = w_cov_matrix['TP9'].append(pd.Series(5))
w_cov_matrix

,TP9,AF7,AF8,TP10
TP9,6104.029091,2558.714800,-2881.880696,9327.740623
AF7,2558.714800,1211.478404,-1158.673931,4054.475958
AF8,-2881.880696,-1158.673931,2151.224463,-4634.389254
TP10,9327.740623,4054.475958,-4634.389254,18078.856431


In [40]:
# Signals variance
w_var = pd.Series(np.diag(w_cov_matrix), index=['variance_0','variance_1','variance_2','variance_3'])
np.diag(w_cov_matrix)

array([ 6104.02909133,  1211.47840428,  2151.22446307, 18078.85643089])

In [8]:
# Signals covariance as pandas Series
w_cov_series = pd.Series({'cov_0':w_cov_matrix.iloc[0,1], 'cov_1':w_cov_matrix.iloc[0,2], 
                          'cov_2':w_cov_matrix.iloc[0,3], 'cov_3':w_cov_matrix.iloc[1,2],
                          'cov_4':w_cov_matrix.iloc[1,3], 'cov_5':w_cov_matrix.iloc[2,3]})
w_cov_series

cov_0    2558.714800
cov_1   -2881.880696
cov_2    9327.740623
cov_3   -1158.673931
cov_4    4054.475958
cov_5   -4634.389254
dtype: float64

In [9]:
# Compute eigen values for covariance matrix
from numpy import linalg

eig = linalg.eigvals(w_cov_matrix)
eig = pd.Series({'eignval_0':eig[0],'eignval_1':eig[1],'eignval_2':eig[2],'eignval_3':eig[3],})
eig

eignval_0    25502.221074
eignval_1     1275.106593
eignval_2      659.006063
eignval_3      109.254660
dtype: float64

In [10]:
# Concatenate all full windows features
result = pd.DataFrame(data=pd.concat([mean,std,w_skew,w_kurtosis,w_var,w_cov_series,eig])).T
result

,mean_0,mean_1,mean_2,mean_3,std_0,std_1,std_2,std_3,skew_0,skew_1,...,cov_0,cov_1,cov_2,cov_3,cov_4,cov_5,eignval_0,eignval_1,eignval_2,eignval_3
0,-68.208684,30.029301,92.912652,-38.366312,78.128286,34.806298,46.381294,134.457638,-0.15739,-0.849355,...,2558.7148,-2881.880696,9327.740623,-1158.673931,4054.475958,-4634.389254,25502.221074,1275.106593,659.006063,109.25466


In [38]:
### HALF WINDOW OPERATIONS ###
wh = np.vsplit(w,2)
# Compute the change in the sample means between 1st and 2nd half-window
d_mean = (wh[1].mean() - wh[0].mean()) / 2
d_mean.rename(index={'TP9':'d_mean_0','AF7':'d_mean_1',
    'AF8':'d_mean_2','TP10':'d_mean_3'}, inplace=True)
# Compute the change in the sample std deviation between 1st and 2nd half-window
d_std = (wh[1].std() - wh[0].std()) / 2
d_std.rename(index={'TP9':'d_std_0','AF7':'d_std_1',
    'AF8':'d_std_2','TP10':'d_std_3'}, inplace=True)
# Compute the change in the sample max between 1st and 2nd half-window
d_max = (wh[1].max() - wh[0].max()) / 2
d_max.rename(index={'TP9':'d_max_0','AF7':'d_max_1',
    'AF8':'d_max_2','TP10':'d_max_3'}, inplace=True)
# Compute the change in the sample min between 1st and 2nd half-window
d_min = (wh[1].min() - wh[0].min()) / 2
d_min.index = [f'hw_d_min_{count}' for count,idx in enumerate(d_min.index)]
d_min

# d_min.rename(index={'TP9':'d_min_0','AF7':'d_min_1',
#     'AF8':'d_min_2','TP10':'d_min_3'}, inplace=True)

#result_h = pd.DataFrame(data=pd.concat([d_mean,d_std,d_max,d_min])).T
#result_h

hw_d_min_0     73.486
hw_d_min_1     50.537
hw_d_min_2    -66.162
hw_d_min_3    130.127
dtype: float64

In [47]:
dfs = []
dfs.append(result)
dfs.append(result_h)
dfs = pd.concat(dfs,axis=0, sort=False)
dfs.reindex()

,mean_0,mean_1,mean_2,mean_3,std_0,std_1,std_2,std_3,skew_0,skew_1,...,d_std_2,d_std_3,d_max_0,d_max_1,d_max_2,d_max_3,d_min_0,d_min_1,d_min_2,d_min_3
0,-68.208684,30.029301,92.912652,-38.366312,78.128286,34.806298,46.381294,134.457638,-0.15739,-0.849355,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.773428,-10.72077,53.223,10.7425,-31.7385,83.252,73.486,50.537,-66.162,130.127


In [35]:
# Compute mean for each quarter window
wq_mean = [x.mean() for x in wq] # 4 features per channel

wq_d_mean = []
# Compute the change in the mean for each quarter window
wq_d_mean.append((wq_mean[1] - wq_mean[0]) / 2)
wq_d_mean.append((wq_mean[2] - wq_mean[0]) / 2)
wq_d_mean.append((wq_mean[3] - wq_mean[0]) / 2)
wq_d_mean.append((wq_mean[2] - wq_mean[1]) / 2)
wq_d_mean.append((wq_mean[3] - wq_mean[1]) / 2)
wq_d_mean.append((wq_mean[3] - wq_mean[2]) / 2)

wq_mean = pd.DataFrame(pd.concat(wq_mean, sort=False))
wq_mean.index = [f'qw_mean_{count}' for count,col in enumerate(wq_mean.index)]

wq_d_mean = pd.DataFrame(pd.concat(wq_d_mean, sort=False))
wq_d_mean.index = [f'qw_d_mean_{count}' for count,col in enumerate(wq_d_mean.index)]

type(pd.concat([wq_mean, wq_d_mean],axis=0,sort=False).T)

pandas.core.frame.DataFrame

In [31]:
# Compute max and min for each quarter window
wq_max = [x.max() for x in wq]
wq_min = [x.min() for x in wq]

wq_d_max.append((wq_mean[1] - wq_mean[0]) / 2)
wq_d_max.append((wq_mean[2] - wq_mean[0]) / 2)
wq_d_max.append((wq_mean[3] - wq_mean[0]) / 2)
wq_d_max.append((wq_mean[2] - wq_mean[1]) / 2)
wq_d_max.append((wq_mean[3] - wq_mean[1]) / 2)
wq_d_max.append((wq_mean[3] - wq_mean[2]) / 2)

[TP9    -133.301
 AF7      11.230
 AF8     164.551
 TP10    -81.543
 dtype: float64, TP9     -49.805
 AF7      49.316
 AF8     135.254
 TP10     42.969
 dtype: float64, TP9       8.301
 AF7      69.336
 AF8     101.074
 TP10    123.047
 dtype: float64, TP9      56.641
 AF7      70.801
 AF8      91.797
 TP10    209.473
 dtype: float64]

In [15]:
# Advance window
start += OVERLAP
end = start + 1